In [19]:
# --- Imports and path setup ---
import sys
import os

# Ensure local package is importable
sys.path.append("./")

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_cleaned.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

CSV exists: True
Output dir: ../twilight_outputs


In [20]:
# ---- User-editable parameters ----

# Date range (UTC)
START_DATE = "2021-01-01"
END_DATE = "2021-01-20"

# -- Site --
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

# -- Visibility --
MIN_ALT_DEG = 20.0
TWILIGHT_SUN_ALT_MIN_DEG = -15.0  # minimum altitude for twilight
TWILIGHT_SUN_ALT_MAX_DEG = -5.0  # maximum altitude for twilight

# -- Filters and hardware --
FILTERS = ["g", "r", "i", "z"]
CAROUSEL_CAPACITY = 5
FILTER_CHANGE_S = 120.0
READOUT_S = 2.0
FILTER_CHANGE_TIME_S = None  # legacy alias
READOUT_TIME_S = None  # legacy alias
INTER_EXPOSURE_MIN_S = 15.0
EXPOSURE_BY_FILTER = {"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0}

#cadence knobs
CADENCE_ENABLE = True    #Activates cadence-based filtering and bonus
CADENCE_PER_FILTER = True     #Only tracks cadence per filter (different filters will have separate revisit histories)
CADENCE_DAYS_TARGET = 3.0     #Target days between revisits per filter (hope > then this). gating prevents early observations unless allowed by jitter.
CADENCE_JITTER_DAYS = 0.25     # Days allows revisits slightly before target
CADENCE_DAYS_TOLERANCE = 1  #Window around target for KPI reporting. Observations within ±tolerance count as “on cadence.”
CADENCE_BONUS_SIGMA_DAYS = 0.5  # Standard deviation for Gaussian bonus calculation
CADENCE_BONUS_WEIGHT = 0.25  # Weight for bonus in priority calculation

# Allow colors in one visit (if you want them)
MAX_FILTERS_PER_VISIT = 3
ALLOW_FILTER_CHANGES_IN_TWILIGHT = True  # was False
START_FILTER = FILTERS[0]
SUN_ALT_POLICY = [
    (-18.0, -15.0, ["y", "z", "i"]),
    (-15.0, -12.0, ["z", "i", "r"]),
    (-12.0, 0.0, ["i", "z", "y"]),
]

# -- Slew model --
SLEW_SMALL_DEG = 3.5
SLEW_SMALL_TIME_S = 4.0
SLEW_RATE_DEG_PER_S = 5.25
SLEW_SETTLE_S = 1.0

# -- Moon --
MIN_MOON_SEP_BY_FILTER = {
    "u": 80.0,
    "g": 50.0,
    "r": 35.0,
    "i": 30.0,
    "z": 25.0,
    "y": 20.0,
}
REQUIRE_SINGLE_TIME_FOR_ALL = True

# -- Time caps --
PER_SN_CAP_S = 120.0
MORNING_CAP_S = 'auto'
EVENING_CAP_S = 'auto'
TWILIGHT_STEP_MIN = 2
MAX_SN_PER_NIGHT = 100

# -- Priority strategy --
PRIORITY_STRATEGY = "unique_first"  # or "lc"
HYBRID_DETECTIONS = 2
HYBRID_EXPOSURE = 300.0
LC_DETECTIONS = 5
LC_EXPOSURE = 300.0

# -- Photometry / sky --
PIXEL_SCALE_ARCSEC = 0.2
ZPT1S = None
K_M = None
FWHM_EFF = None
READ_NOISE_E = 6.0
GAIN_E_PER_ADU = 1.0
ZPT_ERR_MAG = 0.01
DARK_SKY_MAG = None
TWILIGHT_DELTA_MAG = 2.5  # fallback if rubin_sim.skybrightness unavailable

# -- SIMLIB output --
SIMLIB_SURVEY = "LSST"
SIMLIB_FILTERS = "grizy"
SIMLIB_PIXSIZE = 0.2
SIMLIB_NPE_PIXEL_SATURATE = 1.0e6
SIMLIB_PHOTFLAG_SATURATE = 4096
SIMLIB_PSF_UNIT = "arcsec"

# -- Miscellaneous --
TYPICAL_DAYS_BY_TYPE = {
    "Ia": 70,
    "II-P": 100,
    "II-L": 70,
    "IIn": 120,
    "IIb": 70,
    "Ib": 60,
    "Ic": 60,
}
DEFAULT_TYPICAL_DAYS = 60
RA_COL = None
DEC_COL = None
DISC_COL = None
NAME_COL = None
TYPE_COL = None


In [21]:
# ---- User-editable parameters ----
PRIORITY_STRATEGY2 = "unique_first"
SIMLIB_OUT2 = "unique_first.simlib"  # any desired filename
cfg2 = PlannerConfig(
    lat_deg=LAT_DEG,
    lon_deg=LON_DEG,
    height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    twilight_sun_alt_min_deg=TWILIGHT_SUN_ALT_MIN_DEG,
    twilight_sun_alt_max_deg=TWILIGHT_SUN_ALT_MAX_DEG,
    filters=FILTERS,
    carousel_capacity=CAROUSEL_CAPACITY,
    filter_change_s=FILTER_CHANGE_S,
    readout_s=READOUT_S,
    filter_change_time_s=FILTER_CHANGE_TIME_S,
    readout_time_s=READOUT_TIME_S,
    inter_exposure_min_s=INTER_EXPOSURE_MIN_S,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    max_filters_per_visit=MAX_FILTERS_PER_VISIT,
    start_filter=START_FILTER,
    sun_alt_policy=SUN_ALT_POLICY,
    slew_small_deg=SLEW_SMALL_DEG,
    slew_small_time_s=SLEW_SMALL_TIME_S,
    slew_rate_deg_per_s=SLEW_RATE_DEG_PER_S,
    slew_settle_s=SLEW_SETTLE_S,
    min_moon_sep_by_filter=MIN_MOON_SEP_BY_FILTER,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
    per_sn_cap_s=PER_SN_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    evening_cap_s=EVENING_CAP_S,
    twilight_step_min=TWILIGHT_STEP_MIN,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    cadence_enable=CADENCE_ENABLE,
    cadence_per_filter=CADENCE_PER_FILTER,
    cadence_days_target=CADENCE_DAYS_TARGET,
    cadence_jitter_days=CADENCE_JITTER_DAYS,
    cadence_days_tolerance=CADENCE_DAYS_TOLERANCE,
    cadence_bonus_sigma_days=CADENCE_BONUS_SIGMA_DAYS,
    cadence_bonus_weight=CADENCE_BONUS_WEIGHT,
    hybrid_detections=HYBRID_DETECTIONS,
    hybrid_exposure_s=HYBRID_EXPOSURE,
    lc_detections=LC_DETECTIONS,
    lc_exposure_s=LC_EXPOSURE,
    priority_strategy=PRIORITY_STRATEGY2,
    pixel_scale_arcsec=PIXEL_SCALE_ARCSEC,
    zpt1s=ZPT1S,
    k_m=K_M,
    fwhm_eff=FWHM_EFF,
    read_noise_e=READ_NOISE_E,
    gain_e_per_adu=GAIN_E_PER_ADU,
    zpt_err_mag=ZPT_ERR_MAG,
    dark_sky_mag=DARK_SKY_MAG,
    twilight_delta_mag=TWILIGHT_DELTA_MAG,
    simlib_out=SIMLIB_OUT2,
    simlib_survey=SIMLIB_SURVEY,
    simlib_filters=SIMLIB_FILTERS,
    simlib_pixsize=SIMLIB_PIXSIZE,
    simlib_npe_pixel_saturate=SIMLIB_NPE_PIXEL_SATURATE,
    simlib_photflag_saturate=SIMLIB_PHOTFLAG_SATURATE,
    simlib_psf_unit=SIMLIB_PSF_UNIT,
    typical_days_by_type=TYPICAL_DAYS_BY_TYPE,
    default_typical_days=DEFAULT_TYPICAL_DAYS,
    ra_col=RA_COL,
    dec_col=DEC_COL,
    disc_col=DISC_COL,
    name_col=NAME_COL,
    type_col=TYPE_COL,
    allow_filter_changes_in_twilight=ALLOW_FILTER_CHANGES_IN_TWILIGHT,
)

cfg2

PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, twilight_sun_alt_min_deg=-15.0, twilight_sun_alt_max_deg=-5.0, filters=['g', 'r', 'i', 'z'], carousel_capacity=5, filter_change_s=120.0, readout_s=2.0, inter_exposure_min_s=15.0, filter_change_time_s=None, readout_time_s=None, exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, max_filters_per_visit=3, start_filter='g', sun_alt_policy=[(-18.0, -15.0, ['y', 'z', 'i']), (-15.0, -12.0, ['z', 'i', 'r']), (-12.0, 0.0, ['i', 'z', 'y'])], sun_alt_exposure_ladder=[], slew_small_deg=3.5, slew_small_time_s=4.0, slew_rate_deg_per_s=5.25, slew_settle_s=1.0, min_moon_sep_by_filter={'u': 80.0, 'g': 50.0, 'r': 35.0, 'i': 30.0, 'z': 25.0, 'y': 20.0}, require_single_time_for_all_filters=True, per_sn_cap_s=120.0, morning_cap_s='auto', evening_cap_s='auto', morning_twilight=None, evening_twilight=None, twilight_step_min=2, max_sn_per_night=100, hybrid_detections=2, hybrid_exposure_s=300.0, lc_detections=5, lc_expo

In [ ]:
# Execute the planner and return data frames
RUN_LABEL = "one_dec"  # label for output files

perSN_df_2, nights_df_2 = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg2,
    run_label=RUN_LABEL,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df_2), "  Nights summary rows:", len(nights_df_2))
perSN_df_2.head(10)